In [20]:
import json
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [34]:
jq_schema_path = '.'
loader = JSONLoader('/home/rdharmani/persona/LT-implicit-memory/full_synthetic_conversation_cc_redial_ultrachat_1.json', jq_schema=jq_schema_path,  text_content=False)
documents = loader.load()

In [18]:
def format_document(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)


In [25]:
# Convert the texts to a list of dictionaries
texts_dict = [text.dict() for text in texts]

# Write the texts to a JSON file
with open('texts.json', 'w') as f:
    json.dump(texts_dict, f)

In [29]:
texts_dict[1]

{'id': None,
 'metadata': {'source': '/home/rdharmani/persona/LT-implicit-memory/full_synthetic_conversation_cc_redial_ultrachat_1.json',
  'seq_num': 1},
 'page_content': 'sports with a creative outlet.\'], \'syn_trait_conv\': "Here is the conversation:\\n\\nSpeaker1: Hi, I\'m looking for some advice on a gift for someone. Do you have any ideas?\\nAssistant: Of course, I\'d be happy to help! Can you tell me a bit more about the person you\'re looking to get a gift for? What\'s their age, interests, etc.?\\n\\nSpeaker1: Well, they\'re an adult, but that\'s about all I can say for now. I want the gift to be something they\'ll really enjoy, but I\'m having trouble coming up with ideas.\\nAssistant: No worries, we can brainstorm together. Are you thinking of something practical, or something more fun and entertaining?\\n\\nSpeaker1: Definitely something fun. I want to bring a smile to their face when they open it.\\nAssistant: That\'s great! Fun gifts are always a good idea. Can you give 

In [ ]:
# # Recursively split the JSON data
# json_chunks = text_splitter.split_json(texts_dict)

# # Convert the chunks to Document objects
# documents = [Document(**chunk) for chunk in json_chunks if chunk]

# # Update the texts variable with the new documents
# texts = documents

AttributeError: 'RecursiveCharacterTextSplitter' object has no attribute 'split_json'

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


/tmp/ipykernel_1755920/780278957.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/data/rdharmani/anaconda3/envs/parsonaRAG/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
texts[0]

Document(metadata={'source': '/home/rdharmani/persona/LT-implicit-memory/full_synthetic_conversation_cc_redial_ultrachat_1.json', 'seq_num': 1}, page_content='[{\'persona\': \'This person loves sports.\', \'reason\': \'They are experiencing persistent back pain due to a herniated disc that needs to be surgically treated.\', \'question\': \'What can I do to occupy my free time in a way that feels fulfilling?\', \'noisy_scenarios\': [\'I can volunteer to coach a local youth sports team to share my passion and give back to the community.\', \'Learning about the science behind athletic performance could be an interesting way to explore the technical side of sports.\', \'Joining a recreational league or club for my favorite sport would allow me to meet like-minded people and stay active.\', \'Starting a sports blog or YouTube channel could be a great way to share my knowledge and opinions with others.\', \'Taking up sports photography would allow me to combine my love of sports with a creat

In [42]:
import os

if 'texts' not in globals():
    with open('/home/rdharmani/persona/LT-implicit-memory/generate_data/texts.json', 'r') as f:
        texts_dict = json.load(f)
    texts = [Document(**text) for text in texts_dict]
    texts = [texts[0]]


In [43]:
db = FAISS.from_documents(texts, embeddings)

In [12]:
db.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search("qux")

In [13]:
docs

[Document(metadata={'source': '/home/rdharmani/persona/LT-implicit-memory/full_synthetic_conversation_cc_redial_ultrachat_1.json', 'seq_num': 1}, page_content='and implications of quantum computing. It\\\'s like my whole understanding of reality was shifted.\\n\\nAssistant: Quantum physics can be mind-bending, indeed! It\\\'s fascinating to learn about the latest advancements and theories. If you\\\'re interested, I can recommend some documentaries on the topic. Sometimes, visual explanations can help solidify our understanding of complex concepts.\\n\\nSpeaker1: That would be amazing, thank you! I\\\'d love to learn more about quantum physics and explore other topics in science and tech.\'], \'cc_noisy_conv\': ["Speaker1: Hey, I found your kitten on the other side of the fence in our yard earlier.\\nAssistant: What?! Thank you so much for bringing her back! I\'ve been looking for her all morning.\\n\\nSpeaker1: No problem. She was just wandering around, but she seemed pretty happy whe

In [ ]:
from langchain_community.chat_models import ChatSambaNovaCloud

chat = ChatSambaNovaCloud(
    sambanova_url = "https://api.sambanova.ai/v1/chat/completions",
    sambanova_api_key = "SAMBA_NOVA_API_KEY",
    model = "Meta-Llama-3.1-405B-Instruct",
    max_tokens = 1024,
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [45]:
from dotenv import load_dotenv
load_dotenv()


query="For each of the conversations answer the given question. "
result=db.similarity_search(query)
# print(result[0].page_content)


llm=chat

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context. I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}
"""
)


#chain
document_chain = create_stuff_documents_chain(llm,prompt) 

retriever = db.as_retriever()


#retriever chain

# from langchain.chains import create_retrieval_chain
# retrieval_chain= create_retrieval_chain(retriever,document_chain)
# response = retrieval_chain.invoke({'input': prompt, 'context':"hvhcv"})
# # Once a query arises it goes to the retriever which interfaces vector store. Then it goes to llm model with prompt and stuff document chain
# print(response['answer'])

In [46]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f89cd9d6970>, search_kwargs={})

In [49]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
print(prompt)
rag_chain = (
    {"context": retriever | format_document, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What can I do to occupy my free time in a way that feels fulfilling?")

/data/rdharmani/anaconda3/envs/parsonaRAG/lib/python3.9/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


'To occupy your free time in a fulfilling way, consider what nourishes you and what you enjoy doing. Reflect on your values and priorities to identify activities that align with them, such as hobbies, spending time with loved ones, or learning a new skill. By making time for these activities, you can create a sense of purpose and fulfillment in your life.'

In [1]:
%pip show langchain

Name: langchain
Version: 0.3.7
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /data/rdharmani/anaconda3/envs/parsonaRAG/lib/python3.9/site-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_text_splitters import RecursiveJsonSplitter

splitter = RecursiveJsonSplitter(max_chunk_size=300)
# Recursively split json data - If you need to access/manipulate the smaller json chunks
json_chunks = splitter.split_json(json_data= 